<h3>가중치 행렬에 대한 gradient 구하기</h3>

In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np

In [2]:
sys.path

['',
 'C:\\Users\\kitcoop\\Anaconda3\\envs\\test\\python36.zip',
 'C:\\Users\\kitcoop\\Anaconda3\\envs\\test\\DLLs',
 'C:\\Users\\kitcoop\\Anaconda3\\envs\\test\\lib',
 'C:\\Users\\kitcoop\\Anaconda3\\envs\\test',
 'C:\\Users\\kitcoop\\Anaconda3\\envs\\test\\lib\\site-packages',
 'C:\\Users\\kitcoop\\Anaconda3\\envs\\test\\lib\\site-packages\\win32',
 'C:\\Users\\kitcoop\\Anaconda3\\envs\\test\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\kitcoop\\Anaconda3\\envs\\test\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\kitcoop\\Anaconda3\\envs\\test\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\kitcoop\\.ipython',
 '..']

In [3]:
def softmax(x):
    if ndim == 2:
        x = x.T
    

In [57]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [58]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [59]:
net = simpleNet()
print(net.W)

[[-0.19648271 -0.94537019  0.40500202]
 [-0.02554468 -1.85250825  0.91510022]]


In [60]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[-0.14087984 -2.23447954  1.06659141]


In [61]:
np.argmax(p) # 소프트맥스 결과 중 가장 큰 값의 인덱스

2

In [62]:
t = np.array([1,0,0])

In [63]:
net.loss(x,t) # input x를 받아 predict -> softmax 후 레이블 t와의 crossentropy 계산

1.4969981650614752

가중치 행렬 W에 대한 gradient 계산

In [66]:
def f(W):
    return net.loss(x,t)

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

input이 아래와 같을 때 
x = [0.6, 0.9]  

W의 gradient 

In [68]:
dW = numerical_gradient(f, net.W)
dW

array([[-0.46571927,  0.01654909,  0.44917019],
       [-0.69857891,  0.02482363,  0.67375528]])